# Hyper-parameter Optimization with RAPIDS + MLflow + Hyperopt

### Introduction

Hyperparameter optimization is the task of picking the values for the hyperparameters of the model that provide the optimal results for the problem, as measured on a test dataset. This is often a crucial step and can help boost the model performance when done correctly. Despite its theoretical importance, HPO has been difficult to implement in practical applications because of the resources needed to run so many distinct training jobs. In this notebook, we explore the combination of RAPIDS, MLflow and Hyperopt to perform HPO on GPU and compare them with the performance of the CPU runs. We want to illustrate that HPO can be performed in an efficient manner with RAPIDS libraries on GPU.
      
### MLFlow

[MLflow](https://mlflow.org/) is used for managing the machine learning lifecycle. It provides a way to track experiements and store data about them. MLflow also provides a way to deploy ML models. We'll make use of MLflow to store information about the models and use the built-in integration of MLflow into Databricks to register and store models.

### Hyperopt

[Hyperopt](http://hyperopt.github.io/hyperopt/) is a library for finding the best hyperparameters for a given objective function. It provides a way to choose the objective function, the search algorithm, the database in which to store the eval points. We'll use this to define the parameter space to search over. 

We'll use FAA flight history data for this demo - the aim is to predict if a given flight will be delayed or not using a target variable `ArrDelayBinary`. We'll use a Random Forest Classifier as the model for the learning task. 

We will compare the performance between the CPU version with scikit-learn to the GPU version with RAPIDS libraries.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import cudf
import cuml

import mlflow
import hyperopt

import numpy as np
import pandas as pd

import mlflow.sklearn
from mlflow.tracking.client import MlflowClient
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

import cuml.ensemble
import cuml.metrics
import cuml.preprocessing.model_selection

import sklearn.ensemble
import sklearn.metrics
import sklearn.model_selection

In [ ]:
# Small utility that times a block of code and prints how long it took to execute

from contextlib import contextmanager
import time

@contextmanager
def timed(name):
    t0 = time.time()
    yield
    t1 = time.time()
    print("..%-24s:  %8.4f" % (name, t1 - t0))

## Experiment set-up
Here we'll define the input max evaluation runs and parallelism. 

Note: To use `MAX_PARALLEL` on DataBricks, make sure your cluster has at least `MAX_PARALLEL` nodes.

In [ ]:
MAX_EVALS = 20
MAX_PARALLEL = 2

# Load the data and take a look

### Data Acquisition

We'll use the cell below to download the data. The `file_name` specifies which of the two available files - `airline_small.parquet` (smaller file) and `airline_20000000.parquet` we want to use. By default, we use the smaller file, but the benchmarks were run with the larger file. You are free to change it for experimentation.

Run the cell below just once in the cluster to acquire the data and the comment it out for future runs.

In [ ]:
# Read above instructions - RUN ONLY ONCE

from urllib.request import urlretrieve
import os

file_name = 'airline_small.parquet' # NOTE: Change to airline_20000000.parquet to use a larger dataset

data_dir = "/_dbfs_path/" # NOTE: Change to DBFS path where you want to save the file
INPUT_FILE = os.path.join(data_dir, file_name)

if os.path.isfile(parquet_name):
        print(f" > File already exists. Ready to load at {parquet_name}")
else:
    # Ensure folder exists
    os.makedirs(data_dir, exist_ok=True)
        
url = "https://rapidsai-cloud-ml-sample-data.s3-us-west-2.amazonaws.com/" + file_name

urlretrieve(url= url,filename=parquet_name)

print("Completed!")

Let's take a quick peek into the data we will be using. As described before, this is a binary classification problem with the target variable as `ArrDelayBinary`

In [ ]:
df = cudf.read_parquet(INPUT_FILE)
print("Data shape: ", df.shape)
df.head()

# Setting up MLFlow runs

For the experiment to use MLFlow, we will define a training function (one each for CPU and GPU). In these functions, we will see a few mlflow log statements, these help in keep track of the experiment set-up.

In [ ]:
def train_cpu(params, test_set_frac=0.2, registered_model_name=None):
    """
    Train scikit-learn model on the data, and calculate the accuracy on the model.
    This method will be passed to `hyperopt.fmin()`.

    Params:
    params - dict; The range of the HPO space for different parameters (max_depth, max_features, n_estimators)
                   in that order.
    test_set_frac - float; Value between (0,1) for the size of the test set to be used for validation split
    registered_model_name - string; Name under which the best model should be registered with MLFlow.

    Returns:
    dict with fields 'loss' (scalar loss) and 'status' (success/failure status of run)
    """
    max_depth, max_features, n_estimators = params

    with timed("load"):
        df = pd.read_parquet(INPUT_FILE)

    with timed("etl"):
        X = df.drop(["ArrDelayBinary"], axis=1)
        y = df["ArrDelayBinary"].astype('int32')

        X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,
                                                                test_size=test_set_frac,
                                                                random_state=123)

    with timed("fit"):
        mod = sklearn.ensemble.RandomForestClassifier(max_depth=max_depth,
                                             max_features=max_features,
                                             n_estimators=n_estimators,
                                             n_jobs=-1) # Use all available CPUs
        mod.fit(X_train, y_train)

    mlflow.sklearn.log_model(mod, "RF_model_cpu_large_",
                           registered_model_name=registered_model_name)

    with timed("predict"):
        if test_set_frac > 0.0:
            preds = mod.predict(X_test)
            acc = sklearn.metrics.accuracy_score(y_test, preds)
            mlflow.log_metric("accuracy", acc)
        else:
            acc = np.nan

    # Returning -1 * acc because fmin minimizes the "loss" and we want to maximize accuracy.
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
# Example run with a sample parameter value

with timed("sample train skl"):
    result = train_cpu((8, 1.0, 100))

In [ ]:
def train_rapids(params, test_set_frac=0.2, registered_model_name=None):
    """
    Train RAPIDS cuml model on the data, and calculate the accuracy on the model.
    This method will be passed to `hyperopt.fmin()`.

    Params and Return values same as train_cpu
    """

    max_depth, max_features, n_estimators = params

    # Read using cudf
    with timed("read_raw"):
        df = cudf.read_parquet(INPUT_FILE)

    # Converting to dtypes expected by cuml model
    with timed("etl"):
        X = df.drop(["ArrDelayBinary"], axis=1)
        y = df["ArrDelayBinary"].astype('int32')

        # Splitting the data into 80/20 for training and validation
        X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(
                                                    X, y,
                                                    test_size=test_set_frac,
                                                    random_state=123)

    with timed("fit"):
        n_bins = 16
        mod = cuml.ensemble.RandomForestClassifier(max_depth=max_depth,
                                        max_features=max_features,
                                        n_bins=n_bins,
                                        n_estimators=n_estimators)
        mod.fit(X_train, y_train)


    mlflow.sklearn.log_model(mod, "RF_model_GPU_",
                             registered_model_name=registered_model_name)

    with timed("predict"):
        if test_set_frac > 0.0:
            preds = mod.predict(X_test)
            acc = cuml.metrics.accuracy_score(y_test, preds)
            mlflow.log_metric("accuracy", acc)
        else:
            acc = np.nan

    # Returning -1 * acc because fmin minimizes the "loss" and we want to maximize accuracy.
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
# Example run with a sample parameter value
with timed("sample train rapids"):
    result = train_rapids((8, 1.0, 100))

## Configure hyperopt parameter search

Let's define the ranges of the hyperparameter space using HyperOpt. Generally, the larger the ranges the better although, it is useful to keep in mind the limitations posed by the system or cluster you're running on.

In [ ]:
# Shared search parameters
from hyperopt.pyll import scope

search_space = [
        scope.int(hp.quniform('max_depth', 5, 15, 1)),
        hp.uniform('max_features', 0., 1.0),
        scope.int(hp.quniform('n_estimators', 100, 500, 100))
    ]

algo = tpe.suggest
spark.conf.set('spark.task.maxFailures', '1')

## Hyperopt with CPU

In [ ]:
spark_trials = hyperopt.SparkTrials(parallelism=MAX_PARALLEL)
mlflow.end_run() # Close out any run in progress

with mlflow.start_run() as run: 
    mlflow.set_tag("mlflow.runName", "CPU_run")
    best = fmin(
      fn=train_cpu,
      space=search_space,
      algo=algo,
      trials = spark_trials,
      max_evals=MAX_EVALS)
    mlflow.set_tag("best params", str(best))

    # Re-fit the best model on ALL of the data (no test set)
    print(best)
    train_cpu((int(best["max_depth"]), best["max_features"], int(best["n_estimators"])),
            test_set_frac=0.0, registered_model_name="MLFlow_Airline_CPU_large_")

mlflow.end_run()

## Hyperopt with GPU

In [ ]:
# SparkTrials object will automatically log runs to MLFlow in DataBricks
spark_trials = hyperopt.SparkTrials(parallelism=MAX_PARALLEL)
mlflow.end_run() # Close out any run in progress

with mlflow.start_run() as run:
    mlflow.set_tag("mlflow.runName", "GPU_run")
    best = fmin(fn=train_rapids,
      space=search_space,
      algo=algo,
      trials = spark_trials,
      max_evals=MAX_EVALS)
    mlflow.set_tag("best params", str(best))

    # Re-fit the best model on ALL of the data (no test set)
    train_rapids((int(best["max_depth"]), best["max_features"], int(best["n_estimators"])),
               test_set_frac=0.0, registered_model_name="MLFlow_Airline_RAPIDS")
mlflow.end_run()

## Results

On a `p3.2xlarge` cluster on Databricks, the runtimes observed were 24 minutes for GPU and nearly 14 hours for CPU. We illustrate that RAPIDS on GPU can give up to 35x speedups. Hopefully this will make it easier to integrate hyperparameter optimization into your workflow if it can run in a coffee break rather than running overnight!